In [11]:
import numpy as np
from scipy.integrate import solve_ivp
from numpy.fft import fft2, ifft2
from math import pi
import time

In [ ]:
n = 64    # grid points per axis
N = n**2
m = 1     # number of spirals
L = 10
tend = 4
dt = 0.50
beta = 1
D1 = 0.1
D2 = 0.1

#Grid Setup
x2 = np.linspace(-L, L, n + 1)
xspan = x2[:n]
yspan = x2[:n]
tspan = np.arange(0,tend+dt, dt)
print(tspan)

[X,Y]=np.meshgrid(xspan,yspan, indexing='xy')
u=np.tanh(np.sqrt(X**2+Y**2))*np.cos(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2))) # 64x64
v=np.tanh(np.sqrt(X**2+Y**2))*np.sin(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2))) # 64x64

In [ ]:
u_hat = fft2(u).flatten(order='F') #4096 x 1
v_hat = fft2(v).flatten(order='F')#4096 x 1
f_y0 = np.concatenate((np.real(u_hat), np.imag(u_hat), np.real(v_hat), np.imag(v_hat))) # 16384 x 1


## Part a: Spectral Solution

In [ ]:
kx = (np.pi/L)*np.concatenate((np.arange(0,n//2), np.arange(-n//2, 0)))
ky = kx.copy()
kx[0] = 0
ky[0] = 0

[KX, KY] = np.meshgrid(kx,ky, indexing = 'xy')
K = (KX**2 + KY**2)
FLvec = K.reshape(-1, order='F')

def adv_diffrhs_fourier(t, y, fourierLaplacian, N, n, beta, D1, D2):
    # unpack
    y2 = y.flatten(order='F')

    real_ut_hat = y2[0:N]
    imag_ut_hat = y2[N:2*N]
    u_hat = (real_ut_hat + 1j*imag_ut_hat).reshape((n, n), order='F')
    u = np.real(ifft2(u_hat))

    real_vt_hat = y2[2*N:3*N]
    imag_vt_hat = y2[3*N:4*N]
    v_hat = (real_vt_hat + 1j*imag_vt_hat).reshape((n, n), order='F')
    v = np.real(ifft2(v_hat))

    # nonlinear terms in real space
    A_sq = u*u + v*v
    lambd = 1.0 - A_sq
    omega = -beta * A_sq

    nonlin_u_hat = fft2(lambd * u - omega * v).flatten(order='F')
    nonlin_v_hat = fft2(omega * u + lambd * v).flatten(order='F')

    # diffusion in Fourier space
    u_hat_vec = u_hat.flatten(order='F')
    v_hat_vec = v_hat.flatten(order='F')

    ut = nonlin_u_hat - D1 * (fourierLaplacian * u_hat_vec)
    vt = nonlin_v_hat - D2 * (fourierLaplacian * v_hat_vec)

    rhs = np.concatenate((np.real(ut), np.imag(ut), np.real(vt), np.imag(vt)))
    return rhs


sol = solve_ivp(lambda t, y:adv_diffrhs_fourier(t, y, FLvec, N, n, beta, D1, D2),[tspan[0], tspan[-1]], f_y0, t_eval=tspan, method= 'RK45')
t_sol = sol.t 
u_sol = sol.y.T


In [ ]:
print(K.shape)

In [ ]:
real_ut_hat = u_sol[:,0:N] # first 4096
imag_ut_hat = u_sol[:,N:2*N] #second 4096
real_vt_hat = u_sol[:,2*N:3*N] # third 4096
imag_vt_hat = u_sol[:,3*N:4*N] # final 4096
# construct A1 and A2
A1 = np.concatenate((u_sol[:,0:N], u_sol[:,2*N:3*N]), axis=1)
A2 = np.concatenate((u_sol[:,N:2*N], u_sol[:,3*N:4*N]), axis=1)
print(A1)
print(A2)

## Part b: Chebyshev Polynomials

In [12]:
def cheb(N):
    if N == 0:
        return np.array([[0.]]), np.array([1.])
    x = np.cos(pi * np.arange(0, N + 1) / N).reshape(-1, 1)  # column
    c = np.hstack(([2.], np.ones(N - 1), [2.])) * ((-1) ** np.arange(0, N + 1))
    X = np.tile(x, (1, N + 1))
    dX = X - X.T
    C = c.reshape(-1, 1) @ (1.0 / c).reshape(1, -1)
    D = C / (dX + np.eye(N + 1))
    D = D - np.diag(np.sum(D, axis=1))
    return D, x.flatten()

In [13]:
n_cheb = 31
m = 1     # number of spirals
L = 10.0
tend = 4
dt = 0.50
beta = 1
D1 = 0.1
D2 = 0.1

tspan = np.arange(0,tend+dt, dt)

D, x_cheb = cheb(n_cheb-1)
D = (1.0 / L) * D
D_2 = D @ D
D_2[0, :] = 0.0
D_2[-1, :] = 0.0
print(D_2.shape[0])

I = np.eye(n_cheb)
K_cheb = np.kron(D_2, I) + np.kron(I, D_2)
print(K_cheb.shape)

x_phys = L*x_cheb

[Xc,Yc]=np.meshgrid(x_phys,x_phys, indexing='xy')

u_cheb=np.tanh(np.sqrt(Xc**2+Yc**2))*np.cos(m*np.angle(Xc+1j*Yc)-(np.sqrt(Xc**2+Yc**2))) # 31 x 31
u_vec_c = u_cheb.flatten(order='F')
v_cheb=np.tanh(np.sqrt(Xc**2+Yc**2))*np.sin(m*np.angle(Xc+1j*Yc)-(np.sqrt(Xc**2+Yc**2))) # 31 x 31
v_vec_c = v_cheb.flatten(order='F')

init_cheb = np.concatenate((u_vec_c,v_vec_c))


31
(961, 961)


In [14]:
def adv_diffrhs_cheb(t,y, laplace, n_cheb, D1, D2, beta):
    NSQ = n_cheb**2
    u_cheb = y[0:NSQ] #1922 x
    v_cheb = y[NSQ:]
    Asq_c = (u_cheb**2 + v_cheb**2)
    lambd_cheb = 1 - Asq_c
    omega_cheb = -beta*Asq_c

    u_t_cheb = (lambd_cheb * u_cheb - omega_cheb * v_cheb) + D1*(laplace @ u_cheb)
    v_t_cheb = (omega_cheb * u_cheb + lambd_cheb * v_cheb) + D2*(laplace @ v_cheb)

    return np.concatenate([u_t_cheb, v_t_cheb])


solCheb = solve_ivp(lambda t, y:adv_diffrhs_cheb(t, y, K_cheb, n_cheb, D1, D2, beta),[tspan[0], tspan[-1]], init_cheb, t_eval=tspan, method= 'RK45')

u_sol_cheb = solCheb.y.T
t_sol_cheb = solCheb.t

In [15]:
A3 = u_sol_cheb
print(A3.shape)
print(A3)

(9, 1922)
[[ 0.70358468  0.72866166  0.79744529 ...  0.60339126  0.68487385
   0.71061143]
 [ 0.2767684   0.31041693  0.40890462 ...  0.90153377  0.94655769
   0.96093666]
 [-0.21781046 -0.17996952 -0.06803861 ...  0.98493196  0.97895574
   0.97599109]
 ...
 [-0.79682506 -0.81695295 -0.87303039 ... -0.45234802 -0.56606166
  -0.60421009]
 [-0.40960603 -0.44642106 -0.55249147 ... -0.81238663 -0.88765564
  -0.91226252]
 [ 0.07789884  0.03270835 -0.09952284 ... -0.97660577 -0.99274898
  -0.99696127]]
